# 诗歌生成

# 数据处理

In [1]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from keras import layers
from keras import layers, optimizers, datasets
import os


start_token = 'bos'
end_token = 'eos'

def process_dataset(fileName):
    examples = []


    # 获取当前工作目录
    current_directory = os.getcwd()
    print("当前工作目录是：", current_directory)

    # 列出当前目录下所有文件和目录
    contents = os.listdir(current_directory)
    print("当前目录内容包括：")
    for item in contents:
        print(item)
    
    with open(fileName,'r', encoding='UTF-8') as fd:
        for line in fd:
            outs = line.strip().split(':')
            content = ''.join(outs[1:])
            ins = [start_token] + list(content) + [end_token] 
            if len(ins) > 200:
                continue
            examples.append(ins)
            
    counter = collections.Counter()
    for e in examples:
        for w in e:
            counter[w]+=1
    
    sorted_counter = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*sorted_counter)
    words = ('PAD', 'UNK') + words[:len(words)]
    word2id = dict(zip(words, range(len(words))))
    id2word = {word2id[k]:k for k in word2id}
    
    indexed_examples = [[word2id[w] for w in poem]
                        for poem in examples]
    seqlen = [len(e) for e in indexed_examples]
    
    instances = list(zip(indexed_examples, seqlen))
    
    return instances, word2id, id2word

def poem_dataset():
    instances, word2id, id2word = process_dataset('poems.txt')
    ds = tf.data.Dataset.from_generator(lambda: [ins for ins in instances], 
                                            (tf.int64, tf.int64), 
                                            (tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.shuffle(buffer_size=10240)
    ds = ds.padded_batch(100, padded_shapes=(tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.map(lambda x, seqlen: (x[:, :-1], x[:, 1:], seqlen-1))
    return ds, word2id, id2word

# 模型代码， 完成建模代码

In [2]:
class myRNNModel(keras.Model):
    def __init__(self, w2id):
        super(myRNNModel, self).__init__()
        self.v_sz = len(w2id)
        self.embed_layer = tf.keras.layers.Embedding(self.v_sz, 64, 
                                                    batch_input_shape=[None, None])
        
        self.rnncell = tf.keras.layers.SimpleRNNCell(128)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True)
        self.dense = tf.keras.layers.Dense(self.v_sz)
        
    @tf.function
    def call(self, inp_ids):
        '''
        此处完成建模过程，可以参考Learn2Carry
        '''
        input_emb = self.embed_layer(inp_ids)
        rnn_out = self.rnn_layer(input_emb)
        logits = self.dense(rnn_out)
        return logits

    
    @tf.function
    def get_next_token(self, x, state):
        '''
        shape(x) = [b_sz,] 
        '''
    
        inp_emb = self.embed_layer(x) #shape(b_sz, emb_sz)
        h, state = self.rnncell.call(inp_emb, state) # shape(b_sz, h_sz)
        logits = self.dense(h) # shape(b_sz, v_sz)
        out = tf.argmax(logits, axis=-1)
        return out, state

## 一个计算sequence loss的辅助函数，只需了解用途。

In [3]:
def mkMask(input_tensor, maxLen):
    shape_of_input = tf.shape(input_tensor)
    shape_of_output = tf.concat(axis=0, values=[shape_of_input, [maxLen]])

    oneDtensor = tf.reshape(input_tensor, shape=(-1,))
    flat_mask = tf.sequence_mask(oneDtensor, maxlen=maxLen)
    return tf.reshape(flat_mask, shape_of_output)


def reduce_avg(reduce_target, lengths, dim):
    """
    Args:
        reduce_target : shape(d_0, d_1,..,d_dim, .., d_k)
        lengths : shape(d0, .., d_(dim-1))
        dim : which dimension to average, should be a python number
    """
    shape_of_lengths = lengths.get_shape()
    shape_of_target = reduce_target.get_shape()
    if len(shape_of_lengths) != dim:
        raise ValueError(('Second input tensor should be rank %d, ' +
                         'while it got rank %d') % (dim, len(shape_of_lengths)))
    if len(shape_of_target) < dim+1 :
        raise ValueError(('First input tensor should be at least rank %d, ' +
                         'while it got rank %d') % (dim+1, len(shape_of_target)))

    rank_diff = len(shape_of_target) - len(shape_of_lengths) - 1
    mxlen = tf.shape(reduce_target)[dim]
    mask = mkMask(lengths, mxlen)
    if rank_diff!=0:
        len_shape = tf.concat(axis=0, values=[tf.shape(lengths), [1]*rank_diff])
        mask_shape = tf.concat(axis=0, values=[tf.shape(mask), [1]*rank_diff])
    else:
        len_shape = tf.shape(lengths)
        mask_shape = tf.shape(mask)
    lengths_reshape = tf.reshape(lengths, shape=len_shape)
    mask = tf.reshape(mask, shape=mask_shape)

    mask_target = reduce_target * tf.cast(mask, dtype=reduce_target.dtype)

    red_sum = tf.reduce_sum(mask_target, axis=[dim], keepdims=False)
    red_avg = red_sum / (tf.cast(lengths_reshape, dtype=tf.float32) + 1e-30)
    return red_avg

# 定义loss函数，定义训练函数

In [4]:
@tf.function
def compute_loss(logits, labels, seqlen):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    losses = reduce_avg(losses, seqlen, dim=1)
    return tf.reduce_mean(losses)

@tf.function
def train_one_step(model, optimizer, x, y, seqlen):
    '''
    完成一步优化过程，可以参考之前做过的模型
    '''
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y, seqlen)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y, seqlen) in enumerate(ds):
        loss = train_one_step(model, optimizer, x, y, seqlen)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy())

    return loss

# 训练优化过程

In [5]:
optimizer = optimizers.Adam(0.0005)
train_ds, word2id, id2word = poem_dataset()
model = myRNNModel(word2id)

for epoch in range(10):
    loss = train(epoch, model, optimizer, train_ds)

当前工作目录是： e:\#03-Homework\6大三下-深度学习\exercise\chap6_RNN
当前目录内容包括：
Learn2Carry-exercise.ipynb
poems.txt
poem_generation_with_RNN-exercise.ipynb
README.md
tangshi.txt
tangshi_for_pytorch
题目要求

epoch 0 : loss 8.819897
epoch 1 : loss 6.5677876
epoch 2 : loss 6.071773
epoch 3 : loss 5.827044
epoch 4 : loss 5.6524377
epoch 5 : loss 5.5664043
epoch 6 : loss 5.4650645
epoch 7 : loss 5.2805104
epoch 8 : loss 5.3039794
epoch 9 : loss 5.206331


# 生成过程

In [17]:
def gen_sentence():
    state = [tf.random.normal(shape=(1, 128), stddev=0.5), tf.random.normal(shape=(1, 128), stddev=0.5)]
    cur_token = tf.constant([word2id['bos']], dtype=tf.int32)
    collect = []
    for _ in range(500):
        cur_token, state = model.get_next_token(cur_token, state)
        collect.append(cur_token.numpy()[0])
    return [id2word[t] for t in collect]

print(''.join(gen_sentence()))

江上江头不可见，不知何处不知君。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得无人事，不得无人不得人。eos有不知何处处，不知何处是君人。eos来不得


In [26]:
def gen_tang_poem(begin_word, max_length=100, max_sentences=10):
    # 假设模型和必要的映射已经被加载和初始化
    state = [tf.random.normal(shape=(1, 128), stddev=0.5),
             tf.random.normal(shape=(1, 128), stddev=0.5)]
    cur_token = tf.constant([word2id[begin_word]], dtype=tf.int32)  # 使用开始字初始化
    poem = [begin_word]
    end_token_id = word2id['eos']  # 假设'eos'为结束符的ID

    sentences_generated = 0  # 已生成的句子数量
    for _ in range(max_length):  # 限制诗的最大长度
        cur_token, state = model.get_next_token(cur_token, state)
        token_id = cur_token.numpy()[0]

        if token_id == end_token_id:
            sentences_generated += 1
            if sentences_generated >= max_sentences:
                break  # 如果生成了足够数量的句子，则结束循环
            else:
                poem.append('\n')  # 用换行符代替'eos'，分隔句子
        else:
            poem.append(id2word.get(token_id, '<unk>'))

    return ''.join(poem)


# 生成不同开头词汇的唐诗
words = ["春", "日", "红", "山", "夜", "湖", "海", "月"]
for word in words:
    print(f"《{word}》")
    print(gen_tang_poem(word))
    print()

《春》
春雨落，月色寒花，一枝。
生不可知，不见君人事。
有不可知，不知何处处。
来不可知，不见君人事。
有不可知，不知何处处。
来不可知，不见君人事。
有不可知，不知何处处。
来不可知，不见君人事。
有不可知

《日》
日来无事不知君。
有无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不得不

《红》
红裳上柳花花落，红叶红花一片花。
上不知人不得，不知何处是君人。
来不得无人事，不得无人不得人。
有不知何处处，不知何处是君人。
来不得无人事，不得无人不得人。
有不知何处处，不知何处是君人。
来不得无

《山》
山涛影，风风吹翠叶，月色寒云落，寒风吹落云。
来无限处，不见白云间。
马无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不

《夜》
夜。
生不得无人事，不得无人不得人。
有不知何处处，不知何处是君人。
来不得无人事，不得无人不得人。
有不知何处处，不知何处是君人。
来不得无人事，不得无人不得人。
有不知何处处，不知何处是君人。
来不

《湖》
湖上。
中无事，不知何处，不知此，不知何处，不知此，何人不可知。
生不可知，不知何处人。
生不可见，不得无人事。
有不可知，不知何处处。
来不可知，不见君人事。
有不可知，不知何处处。
来不可知，不见君

《海》
海滨。
马无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不得不知君。
马无人事，何人不可知。
生无事事，不得不知君。
马

《月》
月，__《式以》）
格之，一为何。
不知，一为人。
中不可知，不知何处，
生不可知。
有何人，不知何处，不知此，不知何处，不知此，何人不可知。
生不可知，不知何处处。
来不可知，不见君人事。
有不可知，

